In [1]:
'hello world'

'hello world'

In [ ]:
n, k = map(int, input().split())
vals = list(map(int, input().split()))

current_themes = {}
for i in range(k):
  current_themes.get(vals[i], 0) += 1

print(current_themes)

In [2]:
import sqlite3
import pandas as pd

# путь к файлу
conn = sqlite3.connect("data.db")

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

           name
0  transactions
1      customer


In [4]:
query = "SELECT * FROM transactions LIMIT 10;"  # пример запроса
df = pd.read_sql(query, conn)

# посмотри на результат
df

,id,customer_id,amount_rur,success_flg,transaction_dttm
0,1,101,500.00,1,2024-03-17 08:30:00
1,2,102,750.25,0,2024-03-17 09:45:00
2,3,103,1000.50,1,2024-03-17 10:20:00
3,4,101,300.75,1,2024-03-17 11:00:00
4,5,104,1200.80,0,2024-03-17 12:30:00
5,6,101,600.75,1,2024-04-17 11:00:00
6,7,101,200.75,1,2024-04-18 11:00:00
7,8,103,1000.50,1,2024-04-17 10:20:00
8,9,102,350.00,0,2024-04-17 09:45:00


In [8]:
customers = pd.read_sql("SELECT * FROM customer;", conn)
transactions = pd.read_sql("SELECT * FROM transactions;", conn)

# Можно посмотреть
display(customers.head())
transactions.head()

,id,name,start_dttm,birth_dt
0,101,Иванов Иван,2024-01-15,1990-05-10
1,102,Петрова Мария,2024-02-20,1985-11-25
2,103,Сидоров Алексей,2024-03-05,1982-08-15
3,104,Козлова Екатерина,2024-01-02,1995-04-30
4,105,Смирнов Дмитрий,2024-03-10,1977-09-20


,id,customer_id,amount_rur,success_flg,transaction_dttm
0,1,101,500.00,1,2024-03-17 08:30:00
1,2,102,750.25,0,2024-03-17 09:45:00
2,3,103,1000.50,1,2024-03-17 10:20:00
3,4,101,300.75,1,2024-03-17 11:00:00
4,5,104,1200.80,0,2024-03-17 12:30:00


In [81]:
q = """
WITH filtered_transactions AS (
    SELECT
        t.id AS transaction_id,
        t.customer_id,
        t.amount_rur,
        t.transaction_dttm,
        c.name AS customer_name,
        ROW_NUMBER() OVER (PARTITION BY t.customer_id ORDER BY t.transaction_dttm) AS rn
    FROM transactions t
    JOIN customer c ON t.customer_id = c.id
    WHERE t.success_flg = 1
      AND t.amount_rur <= 1000000
),

neighbors AS (
    SELECT
        cur.customer_name,
        cur.transaction_id,
        prev.amount_rur AS prev_amount,
        next.amount_rur AS next_amount,
        cur.amount_rur AS amount_rur
    FROM filtered_transactions cur
    JOIN filtered_transactions prev ON cur.customer_id = prev.customer_id AND cur.rn = prev.rn + 1
    JOIN filtered_transactions next ON cur.customer_id = next.customer_id AND cur.rn = next.rn - 1
)

SELECT
    customer_name,
    transaction_id
FROM neighbors
WHERE
    amount_rur > prev_amount
    AND amount_rur > next_amount;
"""

res = pd.read_sql(q, conn)
display(res)

,customer_name,transaction_id
0,Иванов Иван,6


In [77]:
import pandas as pd


def check_person_trans(person_trans):
    ans = pd.DataFrame(columns=["customer_name", "transaction_id"])

    # display(person_trans)
    
    person_trans = person_trans.sort_values("transaction_dttm")
    
    for i in range(len(person_trans) - 2):  # окно шир 3
        window = person_trans.iloc[i:i+3]
        indexes = window.index
        mid_idx = indexes[1]
        
        
        if window.iloc[1]["amount_rur"] > window.iloc[0]["amount_rur"] and window.iloc[1]["amount_rur"] > window.iloc[2]["amount_rur"]:

            name = person_trans.loc[mid_idx]["name"]
            id_trans = person_trans.loc[mid_idx]["id_trans"]

            if window.iloc[1]["success_flg"] == 1 and window.iloc[1]["amount_rur"] <= 1_000_000:

                sample = pd.DataFrame(data=[[name, id_trans]], columns=["customer_name", "transaction_id"])
                # sample = person_trans.loc[mid_idx].loc["name", "id_trans"].to_frame().T
                # display(sample)
                
                ans = pd.concat([ans, sample], ignore_index=True)

    return ans
        
def process(customers, transactions):
    data_merded = pd.merge(
        transactions,
        customers,
        how="left",
        left_on="customer_id",
        right_on="id",
        suffixes=("_trans", "_cust")
    )

    data_grouped_users = data_merded.groupby("customer_id")

    trans_good = data_grouped_users.apply(check_person_trans)

    return trans_good.reset_index(drop=True)

In [90]:
import pandas as pd


def check_person_trans(person_trans):
    ans = pd.DataFrame(columns=["customer_name", "transaction_id"])

    person_trans = person_trans[(person_trans["success_flg"] == 1) & (person_trans["amount_rur"] <= 1_000_000)]

    display(person_trans)
    
    for i in range(1, len(person_trans) - 1):  # окно шир 3
        window = person_trans.iloc[i-1:i+2]
        display(window)
        indexes = window.index
        mid_idx = indexes[1]
        
        
        if window.iloc[1]["amount_rur"] > window.iloc[0]["amount_rur"] and window.iloc[1]["amount_rur"] > window.iloc[2]["amount_rur"]:

            name = person_trans.loc[mid_idx]["name"]
            id_trans = person_trans.loc[mid_idx]["id_trans"]

            if window.iloc[1]["success_flg"] == 1 and window.iloc[1]["amount_rur"] <= 1_000_000:

                sample = pd.DataFrame(data=[[name, id_trans]], columns=["customer_name", "transaction_id"])
                # sample = person_trans.loc[mid_idx].loc["name", "id_trans"].to_frame().T
                # display(sample)
                
                ans = pd.concat([ans, sample], ignore_index=True)

    return ans
        
def process(customers, transactions):
    data_merded = pd.merge(
        transactions,
        customers,
        how="left",
        left_on="customer_id",
        right_on="id",
        suffixes=("_trans", "_cust")
    )

    data_grouped_users = data_merded.groupby("customer_id")

    trans_good = data_grouped_users.apply(check_person_trans)

    return trans_good.reset_index(drop=True)

In [91]:
process(customers, transactions).head()

,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt
0,1,101,500.00,1,2024-03-17 08:30:00,101,Иванов Иван,2024-01-15,1990-05-10
3,4,101,300.75,1,2024-03-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10
5,6,101,600.75,1,2024-04-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10
6,7,101,200.75,1,2024-04-18 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10


,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt
0,1,101,500.00,1,2024-03-17 08:30:00,101,Иванов Иван,2024-01-15,1990-05-10
3,4,101,300.75,1,2024-03-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10
5,6,101,600.75,1,2024-04-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10


,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt
3,4,101,300.75,1,2024-03-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10
5,6,101,600.75,1,2024-04-17 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10
6,7,101,200.75,1,2024-04-18 11:00:00,101,Иванов Иван,2024-01-15,1990-05-10


,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt


,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt
2,3,103,1000.5,1,2024-03-17 10:20:00,103,Сидоров Алексей,2024-03-05,1982-08-15
7,8,103,1000.5,1,2024-04-17 10:20:00,103,Сидоров Алексей,2024-03-05,1982-08-15


,id_trans,customer_id,amount_rur,success_flg,transaction_dttm,id_cust,name,start_dttm,birth_dt


/var/folders/bc/z2tbr02j71z_j79b47m1vkp80000gn/T/ipykernel_3731/3986694391.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trans_good = data_grouped_users.apply(check_person_trans)


,customer_name,transaction_id
0,Иванов Иван,6


In [93]:
conn = sqlite3.connect("moderation.db")

verdicts = pd.read_sql("SELECT * FROM verdicts", conn)
print(verdicts)

       index  banner_id     uid verdict
0          0   57421551   51567      No
1          1   38788087   51567      No
2          2   27309982   51567      No
3          3   29100381   51567      No
4          4   72302555   51567     Yes
...      ...        ...     ...     ...
16043  16043   32418988  749066     Yes
16044  16044   32418980  749066     Yes
16045  16045   32418998  749066     Yes
16046  16046   32418982  749066     Yes
16047  16047   32418989  749066     Yes

[16048 rows x 4 columns]


In [144]:
import pandas as pd

def get_sample(df):
    verdict_each_banner = df.groupby('banner_id')['verdict'].value_counts()
    vrdicts_each = verdict_each_banner.unstack()
    
    # (No / (Yes + No))
    vrdicts_each['no_share'] = vrdicts_each['No'] / (vrdicts_each['Yes'] + vrdicts_each['No'])
    
    top_banners = vrdicts_each.sort_values('no_share', ascending=False).head(100)
    return top_banners.reset_index()[["banner_id"]]

In [146]:
get_sample(verdicts)

verdict,banner_id
0,77280121
1,23914
2,30922
3,41955
4,43646
...,...
95,463937
96,463938
97,463939
98,463940


In [148]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 8.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [147]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn import preprocessing



def main():
    train = pd.read_csv('train.csv')
    X_train = train.drop('y', axis=1)
    y_train = train['y']
    
    X_test = pd.read_csv('test_x.csv')

    normalizer = preprocessing.StandardScaler()

    normalizer.fit(X_train)
    
    X_train_norm = pd.DataFrame(
        data=normalizer.transform(X_train),
        index=X_train.index,
        columns=X_train.columns,
    )

    X_test_norm = pd.DataFrame(
        data=normalizer.transform(X_test),
        index=X_test.index,
        columns=X_test.columns,
    )
    
    model = HistGradientBoostingRegressor(
        learning_rate=0.06,
        max_iter=1_500,
        max_depth=7,
        l2_regularization=0.3,
    )
    
    model.fit(X_train_norm, y_train)

    predictions = model.predict(X_test_norm)

    pd.Series(predictions, name='y').to_csv('test_y.csv', index=False)


if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'catboost'

In [159]:
train = pd.read_csv('train.csv')
data = train.drop('y', axis=1)
y = train['y']

In [160]:
from sklearn.impute import SimpleImputer

mis_replacer = SimpleImputer(strategy="mean")

X_no_miss = pd.DataFrame(
    data=mis_replacer.fit_transform(data),
    columns=data.columns
).reset_index(drop=True)

X_no_miss.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20
0,-2.509198,9.014286,4.639879,1.973170,-6.879627,-6.880110,-8.838328,7.323523,2.022300,4.161452,...,9.398197,6.648853,-5.753218,-6.363501,-6.331910,-3.915155,0.495129,-1.361100,-4.175417,2.237058
1,-7.210123,-4.157107,-2.672763,-0.878600,5.703519,-6.006524,0.284689,1.848291,-9.070992,2.150897,...,-8.698968,8.977711,9.312641,6.167947,-3.907725,-8.046558,3.684661,-1.196950,-7.559235,-0.096462
2,-9.312230,8.186408,-4.824400,3.250446,-3.765778,0.401360,0.934206,-6.302911,9.391693,5.502656,...,7.896547,1.958000,8.437485,-8.230150,-6.080343,-9.095454,-3.493393,-2.226454,-4.573019,6.574750
3,-2.864933,-4.381310,0.853922,-7.181516,6.043940,-8.508987,9.737739,5.444895,-6.025686,-9.889558,...,4.137147,4.580143,5.425407,-8.519107,-2.830685,-7.682619,7.262069,2.465963,-3.382040,-8.728833
4,-3.780354,-3.496334,4.592124,2.751149,7.744255,-0.555701,-7.608115,4.264896,5.215701,1.225544,...,-0.124088,0.454657,-1.449180,-9.491617,-7.842171,-9.371416,2.728208,-3.712880,0.171414,8.151329


In [161]:
from sklearn import preprocessing

normalizer = preprocessing.StandardScaler()
X_no_miss_norm = pd.DataFrame(
    data=normalizer.fit_transform(X_no_miss),
    index=X_no_miss.index,
    columns=X_no_miss.columns,
)

In [162]:
X_no_miss_norm.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20
0,-0.439425,1.564953,0.793459,0.350665,-1.175040,-1.200506,-1.541778,1.271423,0.335312,0.695921,...,1.640096,1.161473,-0.991918,-1.109412,-1.094968,-0.682627,0.101774,-0.248612,-0.719032,0.385465
1,-1.259615,-0.719355,-0.465935,-0.144288,1.003891,-1.048786,0.046507,0.323833,-1.586314,0.351462,...,-1.484217,1.565657,1.616469,1.060923,-0.675738,-1.399816,0.652903,-0.220172,-1.304317,-0.021680
2,-1.626378,1.421375,-0.836493,0.572349,-0.635837,0.064102,0.159586,-1.086883,1.611869,0.925704,...,1.380849,0.347353,1.464951,-1.432699,-1.051463,-1.581898,-0.587414,-0.398541,-0.787804,1.142291
3,-0.501492,-0.758239,0.141436,-1.238223,1.062839,-1.483401,1.692250,0.946292,-1.058794,-1.711376,...,0.731823,0.802438,0.943463,-1.482744,-0.489479,-1.336638,1.271053,0.414453,-0.581805,-1.527828
4,-0.661209,-0.604758,0.785235,0.485691,1.357271,-0.102115,-1.327602,0.742071,0.888486,0.192925,...,-0.003841,0.086441,-0.246750,-1.651174,-1.356148,-1.629804,0.487634,-0.656075,0.032821,1.417367


In [203]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_no_miss_norm, y, test_size=0.65
)

In [204]:
X_no_miss_norm.var()

x_0     1.0001
x_1     1.0001
x_2     1.0001
x_3     1.0001
x_4     1.0001
x_5     1.0001
x_6     1.0001
x_7     1.0001
x_8     1.0001
x_9     1.0001
x_10    1.0001
x_11    1.0001
x_12    1.0001
x_13    1.0001
x_14    1.0001
x_15    1.0001
x_16    1.0001
x_17    1.0001
x_18    1.0001
x_19    1.0001
x_20    1.0001
dtype: float64

In [227]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor

regressor = SGDRegressor(
    max_iter=1_000_000_000, 
    tol=1e-3,
    loss="squared_error",
    penalty="l2",
    alpha=0.001,
    
)

regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)

display(mean_absolute_error(predictions, y_test))


model = CatBoostRegressor(
    verbose=False,
    random_seed=42
)

model.fit(X_train, y_train)
predictions_2 = model.predict(X_test)

display(mean_absolute_error(predictions_2, y_test))


model = HistGradientBoostingRegressor(
    learning_rate=0.06,
    max_iter=1_500,
    max_depth=7,
    l2_regularization=0.3,
)

# Обучение
model.fit(X_train, y_train)

# Предсказание
predictions_3 = model.predict(X_test)
mean_absolute_error(predictions_3, y_test)

16.229043855023402

9.494981524642483

10.286961090681476

In [177]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error

def stochastic_gradient_descent(
    w_init: np.ndarray,
    X: np.ndarray,
    y: np.ndarray,
    loss,
    lr: float,
    batch_size: int,
    p: float,
    n_iterations: int = 1000,
):
    """
    Функция градиентного спуска
    :param w_init: np.ndarray размера (n_feratures,) - начальное значение вектора весов
    :param X: np.ndarray размера (n_objects, n_features) - матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) - вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать градиенты при помощи loss.calc_grad(X, y, w)
    :param lr: float - параметр величины шага, на который нужно домножать градиент
    :param batch_size: int - размер подвыборки, которую нужно семплировать на каждом шаге
    :param p: float - значение степени в формуле затухания длины шага
    :param n_iterations: int - сколько итераций делать
    :return: Список из n_iterations объектов np.ndarray размера (n_features,) - история весов на каждом шаге
    """
    gradient_descent_story = np.zeros((n_iterations+1, w_init.shape[0]))
    gradient_descent_story[0] = w_init
    l = y.shape[0]

    for i in np.arange(1, n_iterations+1):

        # чуть изменил и сделал как в лекции: последовательно перебираем все объекты
        batch_indexes = [ i % l for i in range((i-1) * batch_size - 1, i*batch_size - 1) ]
        batch = X.iloc[batch_indexes]

        k = lr * (1 / (1+i))**p

        dash = k * loss.calc_grad(batch, y[batch_indexes], gradient_descent_story[i-1])
        gradient_descent_story[i] = gradient_descent_story[i-1] - dash

    return gradient_descent_story

class MAELoss:
    def __init__(self):
        return self
        
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса (MAE)
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число — значение функции потерь (MAE)
        """
        return (np.abs(np.dot(X, w) - y)).sum() / y.shape[0]

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса (MAE) по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        diff = np.dot(X, w) - y
        sign = np.sign(diff)
        return np.dot(np.transpose(X), sign) / y.shape[0]


class LinearRegression:
    def __init__(self, loss, lr: float = 0.1) -> None:
        self.loss = loss
        self.lr = lr

    def fit(self, X: np.ndarray, y: np.ndarray) -> "LinearRegression":
        X = np.asarray(X)
        y = np.asarray(y)
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        w = stochastic_gradient_descent(
            w_init=np.ones([X.shape[1], ]),
            X=X_train,
            y=y_train,
            loss=mean_absolute_error,
            lr=0.01,
            batch_size=10,
            p=0.1
        )[-1]

        self.w = w

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        # Проверяем, что регрессия обучена, то есть, что был вызван fit и в нём был установлен атрибут self.w
        assert hasattr(self, "w"), "Linear regression must be fitted first"
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        ans = X.dot(self.w)
        return ans


In [178]:
loss = MAELoss()

linear_regression = LinearRegression(loss)

linear_regression.fit(X_train, y_train)
y_train_pred = linear_regression.predict(X_train)
mean_squared_error(y_train_pred, y_train)


TypeError: __init__() should return None, not 'MAELoss'

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor


def main():
    train = pd.read_csv('train.csv')
    X_train = train.drop('y', axis=1)
    y_train = train['y']

    X_test = pd.read_csv('test_x.csv')

    model = CatBoostRegressor(
        verbose=False,
        random_seed=42
    )

    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    pd.Series(predictions, name='y').to_csv('test_y.csv', index=False)


if __name__ == '__main__':
    main()
